# Question 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import time
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
%matplotlib inline

In [ ]:
#downloading the auto data set and viewing its description
input_file = "Auto.csv"
df = pd.read_csv(input_file)
df.info()

In [ ]:
#creating a binary variable(0,1), named mpg01, based on the median mpg
median_mpg = df["mpg"].median()
df["mpg01"] = (df["mpg"] >= df["mpg"].median()).astype(int)
print(df)

In [ ]:
#exploring the data graphically using boxplots to view any associations
#box-plots shows that mpg01 is correlated with cylinder, displacement, and origin 
plt.figure(figsize=(2,2))
c = sns.boxplot(x=df["mpg01"], y=df["cylinder"], palette="Blues", width=0.1);
plt.show(c)
plt.figure(figsize=(2,2))
d = sns.boxplot(x=df["mpg01"], y=df["displacement"], palette="Accent", width=0.1);
plt.show(d)
plt.figure(figsize=(2,2))
h = sns.boxplot(x=df["mpg01"], y=df["horsepower"], palette="Accent", width=0.1);
plt.show(h)
plt.figure(figsize=(2,2))
w = sns.boxplot(x=df["mpg01"], y=df["weight"], palette="coolwarm", width=0.1);
plt.show(w)
plt.figure(figsize=(2,2))
a = sns.boxplot(x=df["mpg01"], y=df["accelaration"], palette="Purples", width=0.1);
plt.show(a)
plt.figure(figsize=(2,2))
y = sns.boxplot(x=df["mpg01"], y=df["year"], palette="Reds", width=0.1);
plt.show(y)
plt.figure(figsize=(2,2))
o = sns.boxplot(x=df["mpg01"], y=df["origin"], palette="autumn", width=0.1);
plt.show(o)

In [ ]:
#exploring associations between mpg01 and other features. 

#this
corr_matrix = df.corr()
corr_matrix["mpg01"].sort_values(ascending=False)

#or this
plt.figure(figsize = (12,6))
sns.heatmap(df.corr(),annot = True) 

#cylinder, displacement, horsepower, weight are codependent
#and highly correlated with mpg/mpg01

In [ ]:
# Separating out the features
features = ['mpg','cylinder','displacement','horsepower', 'weight', 'accelaration','year','origin']
X = df.loc[:, features].values

# Separating out the target
y = df.loc[:,['mpg01']].values

# Standardizing the features
X = StandardScaler().fit_transform(X)
df2 = pd.DataFrame(X, columns = features)
target = pd.DataFrame(y, columns = ['mpg01'])
print(df2)
print(target)

In [ ]:
#splitting the data into training and test set: 80% train, 20% test 
np.random.seed(56)
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]
X_train_set, X_test_set = split_train_test(df2, 0.2)
y_train_set, y_test_set = split_train_test(target, 0.2)
print(len(X_train_set))
print(len(X_test_set))
X_train_set.head()

In [ ]:
#fit data using only two variables that seem most associated with mpg01
features2 = ['cylinder','displacement','horsepower', 'weight']

#lda using sklearn

lda = LinearDiscriminantAnalysis(n_components=1)
X_train_set = X_train_set [features2]
y_train_set = y_train_set['mpg01']
lda.fit_transform(X_train_set,y_train_set)
predicted = lda.predict(X_test_set[features2])
log_mse = mean_squared_error(y_test_set,predicted)
log_rmse = np.sqrt(log_mse)
print("The mse of lda is",log_mse)
print("The rmse of lda is",log_rmse)

#qda using sklearn
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train_set,y_train_set)
predicted2 = qda.predict(X_test_set[features2])
log_mse = mean_squared_error(y_test_set,predicted2)
log_rmse = np.sqrt(log_mse)
print("The mse of qda is",log_mse)
print("The rmse of qda is",log_rmse)
#plt.scatter(X_reduced_lda,y1)


# Question 2 

In [ ]:
#dowloading the data file and getting its description
input_file = "concrete_data.csv"
df3 = pd.read_csv(input_file)
df3.info()

In [ ]:
#using log(x+1) transformation to scale all conditions except age and strength 
df3_condition_variables = ['comp_1','comp_2','comp_3','comp_4','comp_5','comp_6','comp_7']
df3[df3_condition_variables] = np.log(df3[df3_condition_variables]+1)
print(df3)
df3.info

In [ ]:
#using forward selection code to get the best model
def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [ ]:
#running the forward selection function 
model = forward_selected(df3,'strength')
print(model.model.formula)
# sl ~ rk + yr + 1
print(model.rsquared_adj)

In [ ]:
#code hacked from hitters.csv

y = df3.strength
features = ['comp_1','comp_2','comp_3','comp_4','comp_5','comp_6','comp_7','age']
X = df3[features]

#best subset selection code 
def helperfunction(feature_set):
    # Fit model on features and calculate RSS
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

def getBest(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(helperfunction(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model


# This returns a `DataFrame` containing the best model that we generated, along with some extra information about the model. Now we want to call that function for each number of predictors $k$:
models_best = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,9):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")


# Now we have one big `DataFrame` that contains the best models we've generated along with their RSS:


models_best


# If we want to access the details of each model, no problem! We can get a full rundown of a single model using the `summary()` function:


print(models_best.loc[2, "model"].summary())


#r-square is 0.873, two predictors were selected: comp 1 and comp 4